In [ ]:
!pip install torchtext --upgrade
!pip install transformers
!pip install pytorch_lightning

Requirement already up-to-date: torchtext in /usr/local/lib/python3.7/dist-packages (0.9.1)
     |████████████████████████████████| 2.3MB 7.7MB/s 
     |████████████████████████████████| 901kB 43.0MB/s 
     |████████████████████████████████| 3.3MB 50.3MB/s 
     |████████████████████████████████| 808kB 8.0MB/s 
     |████████████████████████████████| 112kB 56.8MB/s 
     |████████████████████████████████| 645kB 47.8MB/s 
     |████████████████████████████████| 10.6MB 34.9MB/s 
     |████████████████████████████████| 276kB 50.0MB/s 
     |████████████████████████████████| 829kB 51.6MB/s 
     |████████████████████████████████| 1.3MB 39.3MB/s 
     |████████████████████████████████| 296kB 50.0MB/s 
     |████████████████████████████████| 143kB 57.8MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=b59020cb808e1c699e3b59de7422d0e2c48ec7ee7df78df5575810ddc17e1359
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886

And download data that we need.

In [ ]:
!git clone https://github.com/SapienzaNLP/nlp2021-hw2

Cloning into 'nlp2021-hw2'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 32 (delta 10), reused 18 (delta 3), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [ ]:
import os

data_folder = os.sep.join(["nlp2021-hw2", "data"])
training_file = [os.sep.join([data_folder, "restaurants_train.json"]), os.sep.join([data_folder, "laptops_train.json"])]
dev_file = [os.sep.join([data_folder, "restaurants_dev.json"]),os.sep.join([data_folder, "laptops_dev.json"])]

In [ ]:
# here go all the imports
import torch
from torch import nn
from torch.utils.data import Dataset
from torchtext import data
from torchtext.vocab import Vectors
from transformers import BertTokenizer, BertModel


from pprint import pprint
from tqdm import tqdm
from torchtext.vocab import Vocab
from collections import Counter
import random
import numpy as np

from typing import *

import json

import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torch.optim as optim

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report 


Set up the seed and deterministic algorithms in order to have reproducible results

In [ ]:
SEED = 96

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True  # will use only deterministic algorithms

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Load data

In [ ]:
class MyDataset(Dataset):

    def __init__(self, 
                 input_file,
                 tokenizer,
                 device="cpu"):
        """
        Args:
            input_file (list of strings or paths): each element is a path to a dataset to be loaded.
            tokeniser BertTokenizer: the tokernizer to be used 
            test Boolean: True if the dataset is used for testing pourposes
            device (string): device where to put tensors (cpu or cuda).
        """

        # load the jsonlines into a list of dictionaries
        self.input_file = input_file
        with open(input_file[0]) as fin:
            data0 = json.load(fin)
        with open(input_file[1]) as fin:
            data1 = json.load(fin)
        
        self.data = data0 + data1

        self.tokenizer = tokenizer
        self.encoded_data = None

        self.classification_labels = {
            "positive" : [1,0,0,0],
            "negative" : [0,1,0,0],
            "neutral" : [0,0,1,0],
            "conflict" : [0,0,0,1],
        }

        self.device = device

        # save the number of samples per class
        self.classes_number = {"negative":0, "positive":0, "conflict":0, "neutral":0}

        # initialize the data
        self.__init_data__()
    
    # remove all the data in order to save space
    def clean_data(self):
        self.data = None
        self.encoded_data = None
                    
    def __init_data__(self):
        self.encoded_data = list()
        
        for i in range(len(self.data)):
            # get the text and lowerize it
            text = self.data[i]["text"].lower()
            
            # for each target create a new train or dev element
            for target in self.data[i]["targets"]:
                
                # get the indeces of the aspect term
                start_idx = target[0][0]
                end_idx   = target[0][1]

                # get the word pieces relative to the text before the aspect term,
                word_pieces_pre = self.tokenizer.tokenize(text[0:start_idx])
                # get the word pieces relative to the aspect term,
                word_pieces_target = self.tokenizer.tokenize(text[start_idx:end_idx])
                # get the word pieces relative to the text after the aspect term,
                word_pieces_post = self.tokenizer.tokenize(text[end_idx:])

                # get the word pieces of the entire sentence
                word_pieces = word_pieces_pre + word_pieces_target + word_pieces_post

                # get the list of indices relative to the word pieces of the 
                # aspect term considered
                target_pieces = list(range(len(word_pieces_pre)+ 1, len(word_pieces_pre + word_pieces_target) + 1))

                # get the ids relative to the word pieces of the sentece
                tokenized = torch.tensor(self.tokenizer.encode(word_pieces))
                 
                # check the label
                label = target[2] 
                assert label in ["negative", "positive", "conflict", "neutral"]
                
                # update count of the class
                self.classes_number[label] += 1

                # associate the label to a OH list
                label = self.classification_labels[label]

                # pass the label to a tensor 
                label = torch.Tensor(label).float().to(self.device)
                self.encoded_data.append({"tokenized":tokenized, 
                                          "target_pieces": target_pieces,
                                          "outputs":label})

   
    def __len__(self):
        return len(self.encoded_data)

    def __getitem__(self, idx):
        return self.encoded_data[idx]



#### Define the torch Model

In [ ]:
class MyModel(nn.Module):
   
    def __init__(self, hparams):
        super(MyModel, self).__init__()
        """
          hparams contains
              dropout
              num_classes: the number of classes we are going to predict
              modality: MAX or AVG
        """

        # initialize bert from pretrained
        self.bert = BertModel.from_pretrained('bert-large-uncased', output_hidden_states=True)

        # initialize dropout and the FC layer
        self.dropout = nn.Dropout(hparams.dropout)
        self.lin1 = torch.nn.Linear(4*1024, hparams.num_classes) 

        self.modality = hparams.modality

    #word_pieces -> list of lists containign the indexes of the parts forming a term
    """
      x: dictionary with 
          input_ids and attention_mask (bert inputs)
      target_pieces: list s.t. for each entry there are the indices of the
                     word pieces of the aspect term considered
    """

    def forward(self, x, target_pieces):
        bert_out = self.bert(x["input_ids"], x["attention_mask"]).hidden_states
        starting_embedding = self.bert.embeddings(x["input_ids"])

        last_h_state_embs = []
        last_but_1_h_state_embs = []
        last_but_2_h_state_embs = []
        last_but_3_h_state_embs = []

        first_layer_embs = []

        # for each sentence-aspect term considered
        # (if a sentence has two aspect terms they will be trained as different
        # entries: the same sentence, but differnet target_pieces)
        for i in range(x["input_ids"].shape[0]):
            if (self.modality == "MAX"):
                last_h_state_embs.append(torch.max(bert_out[-1][i][target_pieces[i][0]:target_pieces[i][-1] + 1], -2).values)
                last_but_1_h_state_embs.append(torch.max(bert_out[-2][i][target_pieces[i][0]:target_pieces[i][-1] + 1], -2).values)
                last_but_2_h_state_embs.append(torch.max(bert_out[-3][i][target_pieces[i][0]:target_pieces[i][-1] + 1], -2).values)
                last_but_3_h_state_embs.append(torch.max(bert_out[-4][i][target_pieces[i][0]:target_pieces[i][-1] + 1], -2).values)
                

                #first_layer_embs.append(torch.max(starting_embedding[i, target_pieces[i][0]:target_pieces[i][-1] + 1, :], -2).values)
            elif (self.modality == "AVG"):
                last_h_state_embs.append(sum(bert_out[-1][i][target_pieces[i][0]:target_pieces[i][-1] + 1])/len(target_pieces[i]))
                last_but_1_h_state_embs.append(sum(bert_out[-1][i][target_pieces[i][0]:target_pieces[i][-1] + 1])/len(target_pieces[i]))
                last_but_2_h_state_embs.append(sum(bert_out[-2][i][target_pieces[i][0]:target_pieces[i][-1] + 1])/len(target_pieces[i]))
                last_but_3_h_state_embs.append(sum(bert_out[-3][i][target_pieces[i][0]:target_pieces[i][-1] + 1])/len(target_pieces[i]))


                #first_layer_embs.append(sum(starting_embedding[i, target_pieces[i][0]:target_pieces[i][-1] + 1, :])/len(target_pieces[i]))
            else:
                raise Exception("No such modality: supported only MAX and AVG")
        
        # stack the data of the different entries
        last_h_state_embs = torch.vstack(last_h_state_embs)
        last_but_1_h_state_embs = torch.vstack(last_but_1_h_state_embs)
        last_but_2_h_state_embs = torch.vstack(last_but_2_h_state_embs)
        last_but_3_h_state_embs = torch.vstack(last_but_3_h_state_embs)
        
        #first_layer_embs = torch.vstack(first_layer_embs)

        # concatenate the abs difference of the starting emb and the output emb 
        # and the output emb
        embs = torch.cat((last_h_state_embs, last_but_1_h_state_embs, last_but_2_h_state_embs, last_but_3_h_state_embs), -1)
        
        #FC
        out = self.lin1(embs)
        
        return out

        

#### Pytorch Lightning Module (Train, Dev and Test code)

In [ ]:

class MyLightningModule(pl.LightningModule):

  
    def __init__(self, hparams, weights=None, *args, **kwargs):
        super(MyLightningModule, self).__init__(*args, **kwargs)
        # save hparams
        self.save_hyperparameters(hparams)
  
        # initialize the loss function as a CrossEntropyLoss for multiclass classification 
        # weight the loss with the tensor weights
        self.loss_function = nn.CrossEntropyLoss(weight=weights)
        
        # initialize the model
        self.model = MyModel(self.hparams)

        self.logits = []
        self.arg_max = []
        self.labels_indices = []

    # This performs a forward pass of the model
    # returns the predicted logits
    """
        x: dictionary containing the input_ids and the attention_mask tensors
           for bert input
        target_pieces: list s.t. for each entry there are the indices of the 
                       word pieces relative to the aspect term considered in that
                       entry
    """
    def forward(self, x, target_pieces):
        logits = self.model(x, target_pieces)

        return logits

    # training step
    """
        batch -> dict containing 
                inputs: dict of input_ids and attention_mask
                outputs: correct labelling
                target_pieces: list s.t. for each entry there are the indices of the 
                       word pieces relative to the aspect term considered in that
                       entry
    """
    def training_step(self, batch, batch_nb):

        inputs = batch['inputs']# dict of input_ids and attention_mask
        labels = batch['outputs']# correct labelling
        target_pieces = batch['target_pieces']# indices of the word pieces of the aspect term

        # get the predicted logits
        logits = self.forward(inputs, target_pieces)
        # apply softmax in order to obtain a probability distribution over the labels
        preds = nn.Softmax(-1)(logits)
        # get the predicted label and the correct one
        arg_max = torch.argmax(preds, dim=-1)
        labels_indices = torch.argmax(labels, dim=-1)
      
        # compute the loss
        loss = self.loss_function(logits, labels_indices)
        
        self.log('train_loss', loss, prog_bar=True)
        print()
        return loss
    

    # validation step -> model in eval state 
    """
        batch -> dict containing 
                inputs: tokenized sentences (sequence of integers) padded
                outputs: correct labelling
                target_pieces: list s.t. for each entry there are the indices of the 
                       word pieces relative to the aspect term considered in that
                       entry
    """
    def validation_step(self, batch, batch_nb):
        inputs = batch['inputs']
        labels = batch['outputs']
        target_pieces = batch['target_pieces']
        

        # get the predicted logits
        logits = self.forward(inputs, target_pieces)
        # apply softmax in order to obtain a probability distribution over the labels
        preds = nn.Softmax(-1)(logits)
        # get the predicted label and the correct one
        arg_max = torch.argmax(preds, dim=-1)
        labels_indices = torch.argmax(labels, dim=-1)

        # compute the loss and the f1 score
        valid_loss = self.loss_function(logits, labels_indices)
        sample_f1 = f1_score(labels_indices.detach().cpu(), arg_max.detach().cpu(), average="macro")
          
        self.log('valid_loss', valid_loss, prog_bar=True)
        self.log('valid_f1', sample_f1, prog_bar=True)


    # test step -> model in eval state
    """
        batch -> dict containing 
                inputs: tokenized sentences (sequence of integers) padded
                outputs: correct labelling
                target_pieces: list s.t. for each entry there are the indices of the 
                       word pieces relative to the aspect term considered in that
                       entry
    """
    def test_step(self, batch, batch_nb):
        inputs = batch['inputs']
        labels = batch['outputs']
        target_pieces = batch['target_pieces']

        # get the predicted logits
        logits = self.forward(inputs, target_pieces)
        self.logits += logits.detach().cpu().tolist()

        # apply softmax in order to obtain a probability distribution over the labels
        preds = nn.Softmax(-1)(logits)
        # get the predicted label and the correct one
        arg_max = torch.argmax(preds, dim=-1)
        labels_indices = torch.argmax(labels, dim=-1)

        self.arg_max += arg_max.detach().cpu().tolist()
        self.labels_indices += labels_indices.detach().cpu().tolist()

        # compute the loss and the f1 score
        f1_per_class = f1_score(self.labels_indices, self.arg_max, average=None)


        self.log('f1 positive', f1_per_class[0], prog_bar=True)
        self.log('f1 negative', f1_per_class[1], prog_bar=True)
        self.log('f1 neutal', f1_per_class[2], prog_bar=True)
        self.log('f1 conflict', f1_per_class[3], prog_bar=True) 

        print(classification_report(self.labels_indices, self.arg_max))

    def configure_optimizers(self):
        return optim.Adam(self.model.parameters(), lr=0.000_01)

In [ ]:
"""
  data is a list of dicts:
             {"tokenized": the encoded word pieces, 
              "target_pieces": the indices of the word pieces of the aspect term considered,
              "outputs": the sentiment for each aspect term
              }

"""

def collate_fn(data):
    X = [entry["tokenized"] for entry in data]

    # get the number of word pieces per sentence
    input_lengths = [entry["tokenized"].shape[0] for entry in data]

    # get the indices of the word pieces of the aspect term considered
    target_pieces = [entry["target_pieces"] for entry in data]

    # pad the data with zeros: the idx of the PAD token for Bert Tokenizer
    X = torch.nn.utils.rnn.pad_sequence(X, batch_first=True, padding_value=0)
    
    # get the labels
    y = torch.vstack([entry["outputs"] for entry in data])

    #build the attention mask as 1s for the word pieces and 0 for the padding
    attention_masks = torch.tensor([[1]*i_len + [0]*(X.shape[1] - i_len) for i_len in input_lengths])
    
    # build the input dictionary for BERT
    input = {"input_ids": X, "attention_mask": attention_masks}

    return {"inputs":input, "outputs":y, "target_pieces":target_pieces}

In [ ]:
class MyLightningDataModule(pl.LightningDataModule):
    def __init__(self, training_file, dev_file, tokenizer, collate_fn, device="cpu"):
        super().__init__()
        self.training_file = training_file
        self.dev_file = dev_file
        self.tokenizer = tokenizer
        self.collate_fn = collate_fn
        self.device = device

        self.setup()

    def setup(self, stage=None):
      self.trainingset = MyDataset(self.training_file, device=self.device, tokenizer=self.tokenizer)
      self.devset = MyDataset(self.dev_file, device=self.device, tokenizer=self.tokenizer)
      self.testset = MyDataset(self.dev_file, device=self.device, tokenizer=self.tokenizer)

          
    def train_dataloader(self):
        return DataLoader(self.trainingset, batch_size=32, collate_fn = self.collate_fn, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.devset, batch_size=256, collate_fn = self.collate_fn)
    
    def test_dataloader(self):
        return DataLoader(self.testset, batch_size=256, collate_fn = self.collate_fn)
    

#### Define Hyperparams and start Training

In [ ]:
hparams = {
            "modality":"AVG",
            "dropout": 0.3,
            "num_classes": 4
           }

In [ ]:
early_stop_callback = EarlyStopping(
   monitor="valid_loss",
   min_delta=0.00,
   patience=3,
   verbose=False,
   mode="min"
)


checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best_model",
    monitor = "valid_loss",
    mode = "min"
)

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

data_module = MyLightningDataModule(training_file, dev_file, tokenizer, collate_fn, device=device)
trainer = pl.Trainer(val_check_interval=1.0, max_epochs=15, gpus=1 if device!=torch.device('cpu') else None, callbacks=[early_stop_callback, checkpoint_callback])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
dataset = MyDataset(training_file, tokenizer)
classes_number = dataset.classes_number

In [ ]:
# build the weights for the loss, so that classes with a lower number of entries
# will have higher cost
total_entries = classes_number["positive"] + classes_number["negative"] + classes_number["neutral"] + classes_number["conflict"]
weights = [(total_entries-classes_number["positive"])/total_entries, (total_entries-classes_number["negative"])/total_entries, (total_entries-classes_number["neutral"])/total_entries, (total_entries-classes_number["conflict"])/total_entries]

In [ ]:
model = MyLightningModule(hparams, weights=torch.tensor(weights))

trainer.fit(model, datamodule=data_module)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_func

### Model Evaluation

In [ ]:
ls -la checkpoints/

total 3919868
drwxr-xr-x 2 root root       4096 Jun 10 15:17 ./
drwxr-xr-x 1 root root       4096 Jun 10 15:17 ../
-rw-r--r-- 1 root root 4013928994 Jun 10 15:18 best_model.ckpt


In [ ]:
# test the overfitted model
trainer.test(model, test_dataloaders=data_module.test_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


              precision    recall  f1-score   support

           0       0.88      0.89      0.89       152
           1       0.75      0.83      0.79        48
           2       0.64      0.57      0.61        47
           3       0.17      0.11      0.13         9

    accuracy                           0.80       256
   macro avg       0.61      0.60      0.60       256
weighted avg       0.79      0.80      0.79       256

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       265
           1       0.75      0.80      0.78       130
           2       0.62      0.53      0.57       102
           3       0.12      0.07      0.09        15

    accuracy                           0.78       512
   macro avg       0.59      0.57      0.58       512
weighted avg       0.76      0.78      0.77       512

              precision    recall  f1-score   support

           0       0.87      0.89      0.88       415
           1       0.

[{'f1 conflict': 0.12851136922836304,
  'f1 negative': 0.7800092101097107,
  'f1 neutal': 0.5765420198440552,
  'f1 positive': 0.8786616325378418}]

In [ ]:
chk_path = "checkpoints/best_model.ckpt"
model = MyLightningModule.load_from_checkpoint(chk_path, weights=torch.tensor(weights))


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# test the model with lower loss
trainer.test(model, test_dataloaders=data_module.test_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.81      0.87      0.84       152
           1       0.63      0.83      0.72        48
           2       0.48      0.32      0.38        47
           3       0.00      0.00      0.00         9

    accuracy                           0.73       256
   macro avg       0.48      0.51      0.49       256
weighted avg       0.69      0.73      0.70       256

              precision    recall  f1-score   support

           0       0.82      0.90      0.86       265
           1       0.70      0.84      0.76       130
           2       0.62      0.40      0.49       102
           3       0.00      0.00      0.00        15

    accuracy                           0.76       512
   macro avg       0.54      0.53      0.53       512
weighted avg       0.73      0.76      0.73       512

              precision    recall  f1-score   support

           0       0.84      0.90      0.87       415
           1       0.

[{'f1 conflict': 0.0,
  'f1 negative': 0.7549207210540771,
  'f1 neutal': 0.4571051597595215,
  'f1 positive': 0.8587276935577393}]

- AVG Last Model
              precision    recall  f1-score   support

           0       0.87      0.88      0.88       546
           1       0.76      0.82      0.79       307
           2       0.63      0.56      0.59       216
           3       0.25      0.16      0.20        25

      acc                              0.79      1094
      M avg        0.63      0.61      0.61      1094
      W avg        0.78      0.79      0.78      1094

- AVG Min Loss
              precision    recall  f1-score   support

           0       0.85      0.89      0.87       546
           1       0.73      0.84      0.78       307
           2       0.60      0.45      0.52       216
           3       0.00      0.00      0.00        25

        acc                            0.77      1094
        M avg      0.54      0.55      0.54      1094
        W avg      0.74      0.77      0.75      1094

- MAX Last Model:
              precision    recall  f1-score   support

           0       0.84      0.89      0.86       546
           1       0.74      0.85      0.79       307
           2       0.62      0.44      0.51       216
           3       0.31      0.16      0.21        25
    
      acc                              0.77      1094
      M avg        0.63      0.58      0.60      1094
      W avg        0.76      0.77      0.76      1094
          
- MAX MinLoss Model:
              precision    recall  f1-score   support

           0       0.86      0.87      0.87       546
           1       0.80      0.67      0.73       307
           2       0.52      0.65      0.58       216
           3       0.19      0.16      0.17        25

      acc                              0.75      1094
      M avg        0.59      0.59      0.59      1094
      W avg        0.76      0.75      0.75      1094

In [ ]:
!ls -la checkpoints/

total 3919868
drwxr-xr-x 2 root root       4096 Jun 10 15:17 .
drwxr-xr-x 1 root root       4096 Jun 10 15:17 ..
-rw-r--r-- 1 root root 4013928994 Jun 10 15:18 best_model.ckpt


In [ ]:
!mv checkpoints/best_model-v1.ckpt ./best_model_A-to_try.ckpt

mv: cannot stat 'checkpoints/best_model-v1.ckpt': No such file or directory
